In [1]:
!git clone https://github.com/jjentaa/aibxesck.git

Cloning into 'aibxesck'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 1), reused 10 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 449.52 KiB | 3.48 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
%cd /content/aibxesck/Khao Man Kai

/content/aibxesck/Khao Man Kai


In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [4]:
%%writefile app.py

import streamlit as st
import groq
import base64
from gtts import gTTS

# Get API Key from https://console.groq.com/keys
client = groq.Groq(
    api_key="XXXXXXXXXXXXXXXXXXXXXXX")

autoplay = True

menu = {
    "ข้าวมันไก่": 65,
    "ไก่สับ": 200,
    "ไก่สับจานใหญ่": 250
}


def get_response(message: str, history) -> str:
    conversation = [
        {
            "role": "system",
            "content": """CREATE YOUR OWN PROMPT"""
        }
    ]

    conversation.extend(history)
    conversation.append({"role": "user", "content": message})

    chat_completion = client.chat.completions.create(
        messages=conversation,
        model="llama-3.1-70b-versatile",
        max_tokens=1000,
        temperature=0.7
    )

    return chat_completion.choices[0].message.content


def tts_wrap(response):
    path = './res.mp3'
    tts = gTTS(response, lang='th')
    tts.save(path)
    # auto_play(tts.stream)
    autoplay_audio3(path)


def autoplay_audio3(file_path):
    file = open(file_path, 'rb').read()
    b64 = base64.b64encode(file).decode()
    if autoplay:
        md = f"""
            <audio id="audioTag" controls autoplay style="width: 99%">
            <source src="data:audio/mp3;base64,{b64}"  type="audio/mpeg" format="audio/mpeg">
            </audio>
            """
    else:
        md = f"""
            <audio id="audioTag" controls style="width: 99%">
            <source src="data:audio/mp3;base64,{b64}"  type="audio/mpeg" format="audio/mpeg">
            </audio>
            """
    st.markdown(
        md,
        unsafe_allow_html=True,
    )


def main():
    st.set_page_config(page_title="chicken rice pra tu nam", page_icon="🍗")
    st.title("ข้าวมันไก่ประตูน้ำ 🍚🍗")

    # first message
    if "messages" not in st.session_state:
        st.session_state.messages = []
        initial_greeting = "สวัสดีครับ ข้าวมันไก่ประตูน้ำยินดีต้อนรับ"
        st.session_state.messages.append(
            {"role": "assistant", "content": initial_greeting})

    for idx, message in enumerate(st.session_state.messages):
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
            if st.button("🔊", key=idx):
                tts_wrap(message["content"])
            # tts_wrap(message["content"])

    if prompt := st.chat_input("What would you like to order?"):
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)
            if st.button("🔊", key=len(st.session_state.messages)):
                tts_wrap(prompt)

        response = get_response(prompt, st.session_state.messages)

        st.session_state.messages.append(
            {"role": "assistant", "content": response})

        with st.chat_message("assistant"):
            st.markdown(response)
            if st.button("🔊", key=len(st.session_state.messages)):
                tts_wrap(response)
            tts_wrap(response)  # autoplay


if __name__ == "__main__":
    main()

Overwriting app.py


In [5]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [16]:
!streamlit run ./app.py &>./logs.txt &

In [17]:
from time import sleep

sleep(1)

with open('./logs.txt', 'r') as f:
    print(f.read())




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.222.92.140:8501




In [13]:
!npx localtunnel --port 8501

your url is: https://whole-jobs-spend.loca.lt
^C
